## Análisis exploratorio datos preparados.

El proposito de este notebook es utilizar el dataset generado en el primer proceso de preparación de datos para realizar analisis exploratorio en los datos, graficas, correlaciones y poder establecer tecnicas para el uso o arreglo de datos.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

### Extracción datos.

In [ ]:
URL_PREPARED_DATA = 'Prepared_Data'
FILE_NAME = 'siniestros-prepared.csv'

In [ ]:
dfsiniestros = pd.read_csv(URL_PREPARED_DATA+'/'+FILE_NAME)
# revisar columnas (4,26) 

### Cantidad de datos.

In [ ]:
dfsiniestros.shape

### Validación de datos.
1.  Validamos los tipos de datos de cada columna.
2.  Validamos la cantidad de valores nan por cada columna.

In [ ]:
dfsiniestros.info()

In [ ]:
for column in dfsiniestros.columns:
    if (dfsiniestros[column].isna().sum()):
        print(f'Columna {column} tienen {dfsiniestros[column].isna().sum()} registros en nan')

### Remplazo de valores vacios *(na)*
- Columna Edad:
    - Se remplaza los valores ```SIN INFORMACION``` en la columna *```EDAD_PROCESADA```*.
    - Se remplaza los valores ```NaN``` en la columna *```EDAD_PROCESADA```*.

In [ ]:
edadlimpia = dfsiniestros[dfsiniestros['EDAD_PROCESADA'] !='SIN INFORMACION'][['EDAD_PROCESADA','SEXO']].dropna()
edadlimpia['EDAD_PROCESADA'] = edadlimpia['EDAD_PROCESADA'].astype(float)

In [ ]:
proedades = edadlimpia.groupby(['SEXO'])['EDAD_PROCESADA'].describe()[['mean','count']].reset_index()
#prueba.columns = prueba.columns.droplevel(0)
print('Promedio Edades Agrupado por SEXO:')
print(proedades)

In [ ]:
print('Moda Edades Agrupado por SEXO:')
moedmasculino = edadlimpia[edadlimpia['SEXO'] == 'MASCULINO'].mode()
print(moedmasculino)
moedfemenino = edadlimpia[edadlimpia['SEXO'] == 'FEMENINO'].mode()
print(moedfemenino)
moednoaplica = edadlimpia[edadlimpia['SEXO'] == 'NO APLICA'].mode()
print(moednoaplica)

#### Conclusion para poblar valores *(na)* campo ```EDAD_PROCESADA```
Como se observo en las anteriores operaciones donde se extrajeron los valores promedio y moda de los valores que actualmente tiene este campo, para contribuir a que la distribucion de los datos se mantengan agrupados por ```SEXO``` se utilizaran los siguientes valores:

- MASCULINO: Se utilizara la moda ```27```
- FEMENINO: Se utilizara el promedio ```38```
- NO APLICA: Se utilizara el promedio ```40```

*Nota: Se toma la decisión de usar estadisticos para la cada ```SEXO``` para que los datos no quedaran planos y nos afectaran la distribucion.*

In [ ]:
edadmasculino = round(moedmasculino['EDAD_PROCESADA'].get(0),0)
edadmasculino

In [ ]:
edadfemenino = round(proedades[proedades['SEXO'] == 'FEMENINO']['mean'].values[0],0)
edadfemenino

In [ ]:
edadnoaplica = round(proedades[proedades['SEXO'] == 'NO APLICA']['mean'].values[0],0)
edadnoaplica

### Eliminación valores
- Columna __EDAD_PROCESADA__  ```SIN INFORMACION```
- Columna __SEXO__  ```NO APLICA```
Total eliminaciones ```4450``` Registros

In [ ]:
dfsiniestros.shape

In [ ]:
# Elimina registros de EDAD_PROCESADA
dfsiniestros.drop(dfsiniestros[(dfsiniestros['EDAD_PROCESADA'].isna() ) | (dfsiniestros['EDAD_PROCESADA'] == 'SIN INFORMACION') ].index,  inplace=True) 

In [ ]:
#Elimina registros de SEXO
dfsiniestros.drop(dfsiniestros[dfsiniestros['SEXO'] == 'NO APLICA'].index, inplace=True)

In [ ]:
dfsiniestros.shape

Remplazo de Valores ```na``` columna ```EDAD_PROCESADA```

In [ ]:
dfsiniestros.loc[((dfsiniestros['SEXO'] == 'MASCULINO') & (dfsiniestros['EDAD_PROCESADA'].isna())),'EDAD_PROCESADA'] = edadmasculino 

In [ ]:
dfsiniestros.loc[((dfsiniestros['SEXO'] == 'FEMENINO') & (dfsiniestros['EDAD_PROCESADA'].isna())),'EDAD_PROCESADA'] = edadfemenino 

In [ ]:
dfsiniestros.loc[((dfsiniestros['SEXO'] == 'NO APLICA') & (dfsiniestros['EDAD_PROCESADA'].isna())),'EDAD_PROCESADA'] = edadnoaplica 

Remplazo de Valores ```'Sin Informacion'``` columna ```EDAD_PROCESADA```

In [ ]:
dfsiniestros.loc[((dfsiniestros['SEXO'] == 'MASCULINO') & (dfsiniestros['EDAD_PROCESADA'] == 'SIN INFORMACION')),'EDAD_PROCESADA'] = edadmasculino 

In [ ]:
dfsiniestros.loc[((dfsiniestros['SEXO'] == 'FEMENINO') & (dfsiniestros['EDAD_PROCESADA'] == 'SIN INFORMACION')),'EDAD_PROCESADA'] = edadfemenino 

In [ ]:
dfsiniestros.loc[((dfsiniestros['SEXO'] == 'NO APLICA') & (dfsiniestros['EDAD_PROCESADA'] == 'SIN INFORMACION')),'EDAD_PROCESADA'] = edadnoaplica 

Validación de tipos de datos en la columna ```EDAD_PROCESADA```

Se convierten todos los tipos de datos de columna ```EDAD_PROCESADA```

In [ ]:
dfsiniestros['EDAD_PROCESADA'] = dfsiniestros['EDAD_PROCESADA'].astype('float')
dfsiniestros['EDAD_PROCESADA'] = dfsiniestros['EDAD_PROCESADA'].astype('int64')

## VALIDAR ELIMINACIONES```TIPO_DISENNO```

In [ ]:
del dfsiniestros['TIPODISENNO']

In [ ]:
del dfsiniestros['TIPOFALLA']

In [ ]:
del dfsiniestros['KEYID']

In [ ]:
del dfsiniestros['Unnamed: 0']

In [ ]:
del dfsiniestros['CON_BICICLETA']

- Columna __LLEVACINTURON__:
    - Se remplaza los valores ```NaN``` en la columna *```LLEVACINTURON```* con ```0``` este siendo una variable de categoria y el  ```0``` significando que para el siniestro no aplica esta categoria.

In [ ]:
dfsiniestros['LLEVACINTURON'].fillna('0', inplace=True)
dfsiniestros['LLEVACINTURON'].value_counts(dropna=False)

- Columna __LLEVACHALECO__:
    - Se remplaza los valores ```NaN``` en la columna __*```LLEVACHALECO```*__ con ```0``` este siendo una variable de categoria y el  ```0``` significando que para el siniestro no aplica esta categoria.

In [ ]:
dfsiniestros['LLEVACHALECO'].fillna('0', inplace=True)
dfsiniestros['LLEVACHALECO'].value_counts(dropna=False)

- Columna __LLEVACASCO__:
    - Se remplaza los valores ```NaN``` en la columna __*```LLEVACASCO```*__ con ```0``` este siendo una variable de categoria y el  ```0``` significando que para el siniestro no aplica esta categoria.

In [ ]:
dfsiniestros['LLEVACASCO'].fillna('0', inplace=True)
dfsiniestros['LLEVACASCO'].value_counts(dropna=False)

- Columna __SERVICIOVEHICULO__:
    - Se remplaza los valores ```NaN``` en la columna __*```SERVICIOVEHICULO```*__ con ```0``` este siendo una variable de categoria y el  ```0``` significando que para el siniestro no aplica esta categoria.

In [ ]:
dfsiniestros['SERVICIOVEHICULO'].fillna('Particular', inplace=True)
dfsiniestros['SERVICIOVEHICULO'].value_counts(dropna=False)

- Columna __VEHICULO_VIAJABA_CLASIFICADO__:
    - Se remplaza los valores ```NaN``` en la columna __*```VEHICULO_VIAJABA_CLASIFICADO```*__ con ```0``` este siendo una variable de categoria y el  ```0``` significando que para el siniestro no aplica esta categoria.

In [ ]:
dfsiniestros['VEHICULO_VIAJABA_CLASIFICADO'].value_counts(dropna=False)

- Columna __CLASEVEHICULO__:
    - Se remplaza los valores ```NaN``` en la columna __*```CLASEVEHICULO```*__ con ```0``` este siendo una variable de categoria y el  ```0``` significando que para el siniestro no aplica esta categoria.

In [ ]:
dfsiniestros['CLASEVEHICULO'].fillna('Automovil', inplace=True)
dfsiniestros['CLASEVEHICULO'].value_counts(dropna=False)

- Columna __CHOQUECODIGO__:
    - Se remplaza los valores ```NaN``` en la columna __*```CHOQUECODIGO```*__ con ```0``` este siendo una variable de categoria y el  ```0``` significando que para el siniestro no aplica esta categoria.

In [ ]:
dfsiniestros['CHOQUECODIGO'].fillna('0', inplace=True)
dfsiniestros['CHOQUECODIGO'].value_counts(dropna=False)

- Columna __MODELOVEHICULO__:
    - Se remplaza los valores ```NaN``` en la columna *```MODELOVEHICULO```* con la moda de los valores correctos.

In [ ]:
dfsiniestros['MODELOVEHICULO'].fillna(2014.0, inplace=True)

In [ ]:
dfsiniestros.info()

### Conversion de tipos de datos por campo y categoricas  __```NUMERICAS```__.
En este paso se realizara lo siguiente:
   1. Por cada campo del ```dfsiniestros``` se convertira al tipo de dato que corresponde por naturaleza. __(Date,Float,Int)__
   2. Las variables categoricas ```alfanumericas``` se convertiran en categoricas ```numericas```.

#### Convertimos el campo __```FECHA```__ en datetime.

In [ ]:
dfsiniestros['FECHA'] = dfsiniestros['FECHA'].astype('datetime64[ns]')

#### Convertimos el campo __```DIA_PROCESADO```__ a categorico numerico de tipo ```INT```.
Para este campo se asignaran los siguientes valores para cada caso:
   1. LUNES
   2. MARTES
   3. MIÉRCOLES
   4. JUEVES
   5. VIERNES
   6. SÁBADO
   7. DOMINGO

In [ ]:
dfsiniestros['DIA_PROCESADO'].replace('LUNES',1,inplace=True)
dfsiniestros['DIA_PROCESADO'].replace('MARTES',2,inplace=True)
dfsiniestros['DIA_PROCESADO'].replace('MIÉRCOLES',3,inplace=True)
dfsiniestros['DIA_PROCESADO'].replace('JUEVES',4,inplace=True)
dfsiniestros['DIA_PROCESADO'].replace('VIERNES',5,inplace=True)
dfsiniestros['DIA_PROCESADO'].replace('SÁBADO',6,inplace=True)
dfsiniestros['DIA_PROCESADO'].replace('DOMINGO',7,inplace=True)

#### Convertimos el campo __```LLEVACINTURON```__ a categorico numerico de tipo ```INT```.
Para este campo se asignaran los siguientes valores para cada caso:

   0. NO APLICA
   1. SI
   2. NO

In [ ]:
dfsiniestros['LLEVACINTURON'].replace('S',1,inplace=True)
dfsiniestros['LLEVACINTURON'].replace('N',2,inplace=True)
dfsiniestros['LLEVACINTURON'] = dfsiniestros['LLEVACINTURON'].astype('int64')

#### Convertimos el campo __```LLEVACHALECO```__ a categorico numerico de tipo ```INT```.
Para este campo se asignaran los siguientes valores para cada caso:

   0. NO APLICA
   1. SI
   2. NO

In [ ]:
dfsiniestros['LLEVACHALECO'].replace('S',1,inplace=True)
dfsiniestros['LLEVACHALECO'].replace('N',2,inplace=True)
dfsiniestros['LLEVACHALECO'] = dfsiniestros['LLEVACHALECO'].astype('int64')

#### Convertimos el campo __```LLEVACASCO```__ a categorico numerico de tipo ```INT```.
Para este campo se asignaran los siguientes valores para cada caso:

   0. NO APLICA
   1. SI
   2. NO

In [ ]:
dfsiniestros['LLEVACASCO'].replace('S',1,inplace=True)
dfsiniestros['LLEVACASCO'].replace('N',2,inplace=True)
dfsiniestros['LLEVACASCO'] = dfsiniestros['LLEVACASCO'].astype('int64')

#### Convertimos el campo __```SEXO```__ a categorico numerico de tipo ```INT```.
Para este campo se asignaran los siguientes valores para cada caso:

   0. NO APLICA
   1. MASCULINO
   2. FEMENINO

In [ ]:
dfsiniestros['SEXO'].replace('MASCULINO',1,inplace=True)
dfsiniestros['SEXO'].replace('FEMENINO',2,inplace=True)
dfsiniestros['SEXO'].replace('NO APLICA',0,inplace=True)
dfsiniestros['SEXO'] = dfsiniestros['SEXO'].astype('int64')

#### Convertimos el campo __```GRAVEDAD_PROCESADA```__ a categorico numerico de tipo ```INT```.
Para este campo se asignaran los siguientes valores para cada caso:

   1. ILESA
   2. HERIDO VALORADO
   3. HERIDO HOSPITALIZADO
   4. MUERTA

In [ ]:
dfsiniestros['GRAVEDAD_PROCESADA'].replace('ILESA',1,inplace=True)
dfsiniestros['GRAVEDAD_PROCESADA'].replace('HERIDO VALORADO',2,inplace=True)
dfsiniestros['GRAVEDAD_PROCESADA'].replace('HERIDO HOSPITALIZADO',3,inplace=True)
dfsiniestros['GRAVEDAD_PROCESADA'].replace('MUERTA',4,inplace=True)
dfsiniestros['GRAVEDAD_PROCESADA'] = dfsiniestros['GRAVEDAD_PROCESADA'].astype('int64')

#### Convertimos el campo __```MODELOVEHICULO```__ en  ```INT```.

In [ ]:
dfsiniestros['MODELOVEHICULO'] = dfsiniestros['MODELOVEHICULO'].astype('int64')

#### Convertimos el campo __```CLASEVEHICULO```__ a categorico numerico de tipo ```INT```.
Para este campo se asignaran los siguientes valores para cada caso:

   1. Automovil
   2. Motocicleta
   3. Camioneta
   4. Bus
   5. Camion, Furgon
   6. Campero
   7. Bicicleta
   8. Microbus
   9. Buseta
   10. Tractocamion
   11. Volqueta
   12. Motocarro
   13. Bicitaxi
   14. Motociclo
   15. Cuatrimoto

In [ ]:
dfsiniestros['CLASEVEHICULO'].replace('Automovil',1,inplace=True)
dfsiniestros['CLASEVEHICULO'].replace('Motocicleta',2,inplace=True)
dfsiniestros['CLASEVEHICULO'].replace('Camioneta',3,inplace=True)
dfsiniestros['CLASEVEHICULO'].replace('Bus',4,inplace=True)
dfsiniestros['CLASEVEHICULO'].replace('Camion, Furgon',5,inplace=True)
dfsiniestros['CLASEVEHICULO'].replace('Campero',6,inplace=True)
dfsiniestros['CLASEVEHICULO'].replace('Bicicleta',7,inplace=True)
dfsiniestros['CLASEVEHICULO'].replace('Microbus',8,inplace=True)
dfsiniestros['CLASEVEHICULO'].replace('Buseta',9,inplace=True)
dfsiniestros['CLASEVEHICULO'].replace('Tractocamion',10,inplace=True)
dfsiniestros['CLASEVEHICULO'].replace('Volqueta',11,inplace=True)
dfsiniestros['CLASEVEHICULO'].replace('Motocarro',12,inplace=True)
dfsiniestros['CLASEVEHICULO'].replace('Bicitaxi',13,inplace=True)
dfsiniestros['CLASEVEHICULO'].replace('Motociclo',14,inplace=True)
dfsiniestros['CLASEVEHICULO'].replace('Cuatrimoto',15,inplace=True)
dfsiniestros['CLASEVEHICULO'] = dfsiniestros['CLASEVEHICULO'].astype('int64')

#### Convertimos el campo __```SERVICIOVEHICULO```__ a categorico numerico de tipo ```INT```.
Para este campo se asignaran los siguientes valores para cada caso:

   1. Particular
   2. Publico
   3. Oficial

In [ ]:
dfsiniestros['SERVICIOVEHICULO'].replace('Particular',1,inplace=True)
dfsiniestros['SERVICIOVEHICULO'].replace('Publico',2,inplace=True)
dfsiniestros['SERVICIOVEHICULO'].replace('Oficial',3,inplace=True)
dfsiniestros['SERVICIOVEHICULO'] = dfsiniestros['SERVICIOVEHICULO'].astype('int64')

#### Convertimos el campo __```POSSESEGURORESPONSABILIDAD```__ a categorico numerico de tipo ```INT```.
Para este campo se asignaran los siguientes valores para cada caso:

   1. SI
   2. NO

In [ ]:
dfsiniestros['POSSESEGURORESPONSABILIDAD'].replace('S',1,inplace=True)
dfsiniestros['POSSESEGURORESPONSABILIDAD'].replace('N',2,inplace=True)
dfsiniestros['POSSESEGURORESPONSABILIDAD'] = dfsiniestros['POSSESEGURORESPONSABILIDAD'].astype('int64')

#### Convertimos el campo __```VEHICULO_VIAJABA_CLASIFICADO```__ a categorico numerico de tipo ```INT```.
Para este campo se asignaran los siguientes valores para cada caso:

   0. SIN INFORMACION
   1. LIVIANO
   2. TRANSPORTE DE PASAJEROS
   3. MOTOCICLETA
   4. TAXI
   5. CARGA
   6. BICICLETA 

In [ ]:
dfsiniestros['VEHICULO_VIAJABA_CLASIFICADO'].replace('SIN INFORMACION',0,inplace=True)
dfsiniestros['VEHICULO_VIAJABA_CLASIFICADO'].replace('LIVIANO',1,inplace=True)
dfsiniestros['VEHICULO_VIAJABA_CLASIFICADO'].replace('TRANSPORTE DE PASAJEROS',2,inplace=True)
dfsiniestros['VEHICULO_VIAJABA_CLASIFICADO'].replace('MOTOCICLETA',3,inplace=True)
dfsiniestros['VEHICULO_VIAJABA_CLASIFICADO'].replace('TAXI',4,inplace=True)
dfsiniestros['VEHICULO_VIAJABA_CLASIFICADO'].replace('CARGA',5,inplace=True)
dfsiniestros['VEHICULO_VIAJABA_CLASIFICADO'].replace('BICICLETA',6,inplace=True)
dfsiniestros['VEHICULO_VIAJABA_CLASIFICADO'] = dfsiniestros['VEHICULO_VIAJABA_CLASIFICADO'].astype('int64')

#### Convertimos el campo __```CON_EMBRIAGUEZ```__ a categorico numerico de tipo ```INT```.
Para este campo se asignaran los siguientes valores para cada caso:

   1. SI
   2. NO

In [ ]:
dfsiniestros['CON_EMBRIAGUEZ'].replace('SI',1,inplace=True)
dfsiniestros['CON_EMBRIAGUEZ'].replace('NO',2,inplace=True)
dfsiniestros['CON_EMBRIAGUEZ'] = dfsiniestros['CON_EMBRIAGUEZ'].astype('int64')

#### Convertimos el campo __```CON_HUECOS```__ a categorico numerico de tipo ```INT```.
Para este campo se asignaran los siguientes valores para cada caso:

   1. SI
   2. NO

In [ ]:
dfsiniestros['CON_HUECOS'].replace('SI',1,inplace=True)
dfsiniestros['CON_HUECOS'].replace('NO',2,inplace=True)
dfsiniestros['CON_HUECOS'] = dfsiniestros['CON_HUECOS'].astype('int64')

#### Convertimos el campo __```CON_PEATON```__ a categorico numerico de tipo ```INT```.
Para este campo se asignaran los siguientes valores para cada caso:
    
   1. SI
   2. NO

In [ ]:
dfsiniestros['CON_PEATON'].replace('SI',1,inplace=True)
dfsiniestros['CON_PEATON'].replace('NO',2,inplace=True)
dfsiniestros['CON_PEATON'] = dfsiniestros['CON_PEATON'].astype('int64')

#### Convertimos el campo __```CHOQUECODIGO```__ en  ```INT```.

In [ ]:
dfsiniestros['CHOQUECODIGO'] = dfsiniestros['CHOQUECODIGO'].astype('int64')

#### Convertimos el campo __```LOCALIDAD```__ a categorico numerico de tipo ```INT```.
Para este campo se asignaran los siguientes valores para cada caso:

   1. KENNEDY               
   2. USAQUEN               
   3. ENGATIVA              
   4. SUBA
   5. FONTIBON
   6. PUENTE ARANDA
   7. CHAPINERO
   8. BARRIOS UNIDOS
   9. TEUSAQUILLO
   10. BOSA
   11. CIUDAD BOLIVAR
   12. LOS MARTIRES
   13. SANTA FE
   14. TUNJUELITO
   15. SAN CRISTOBAL
   16. RAFAEL URIBE URIBE
   17. ANTONIO NARIÑO
   18. USME
   19. CANDELARIA
   20. SUMAPAZ

In [ ]:
dfsiniestros['LOCALIDAD'].replace('KENNEDY',1,inplace=True)
dfsiniestros['LOCALIDAD'].replace('USAQUEN',2,inplace=True)
dfsiniestros['LOCALIDAD'].replace('ENGATIVA',3,inplace=True)
dfsiniestros['LOCALIDAD'].replace('SUBA',4,inplace=True)
dfsiniestros['LOCALIDAD'].replace('FONTIBON',5,inplace=True)
dfsiniestros['LOCALIDAD'].replace('PUENTE ARANDA',6,inplace=True)
dfsiniestros['LOCALIDAD'].replace('CHAPINERO',7,inplace=True)
dfsiniestros['LOCALIDAD'].replace('BARRIOS UNIDOS',8,inplace=True)
dfsiniestros['LOCALIDAD'].replace('TEUSAQUILLO',9,inplace=True)
dfsiniestros['LOCALIDAD'].replace('BOSA',10,inplace=True)
dfsiniestros['LOCALIDAD'].replace('CIUDAD BOLIVAR',11,inplace=True)
dfsiniestros['LOCALIDAD'].replace('LOS MARTIRES',12,inplace=True)
dfsiniestros['LOCALIDAD'].replace('SANTA FE',13,inplace=True)
dfsiniestros['LOCALIDAD'].replace('TUNJUELITO',14,inplace=True)
dfsiniestros['LOCALIDAD'].replace('SAN CRISTOBAL',15,inplace=True)
dfsiniestros['LOCALIDAD'].replace('RAFAEL URIBE URIBE',16,inplace=True)
dfsiniestros['LOCALIDAD'].replace('ANTONIO NARIÑO',17,inplace=True)
dfsiniestros['LOCALIDAD'].replace('USME',18,inplace=True)
dfsiniestros['LOCALIDAD'].replace('CANDELARIA',19,inplace=True)
dfsiniestros['LOCALIDAD'].replace('SUMAPAZ',20,inplace=True)
dfsiniestros['LOCALIDAD'] = dfsiniestros['LOCALIDAD'].astype('int64')

In [ ]:
dfsiniestros['DIRECCION'] = dfsiniestros['DIRECCION'].astype('str')

In [ ]:
dfsiniestros.info()

In [ ]:
dfsiniestros.describe()

### Analisis de datos por medio de graficos.
    En esta sección realizaremos diferentes graficos con el ```dfsiniestros``` para conocer el comportamiento de los datos y realizar cambios en estos.

### Analisis de correlación
#### Heatmap de correlación:
     se realiza un heatmap para analizar la correlación entre los campos.

In [ ]:
#Obtenemos la matriz de correlacion del dataframe
correlacion = dfsiniestros.drop('DIRECCION', axis=1).corr()

# Genera una mascara triangular superior
mask = np.zeros_like(correlacion, dtype = bool)
mask[np.triu_indices_from(mask)] = True

#Configuramos la figura de mathplotlib
f, ax = plt.subplots(figsize=(20,10))

#Creamos el heatmap a partir de la correlacion onbtenida
sns.heatmap(correlacion, mask=mask,square=True, linewidths = .5, ax=ax, cmap='YlOrBr') # cmap='BuPu'
#plt.savefig('prueba')
plt.show()

### Analisis de Edad de personas involucradas en siniestros vial

##### Siniestros por edad 
     analisis de los siniestros que se registran por edad.

In [ ]:
sinrango = dfsiniestros.groupby('EDAD_PROCESADA')['EDAD_PROCESADA'].count().reset_index(name='CANTIDAD')

In [ ]:
plt.figure(figsize=(15,4))
plt.plot(sinrango['EDAD_PROCESADA'],sinrango['CANTIDAD'], 'bo')
plt.xlabel('Edades')
plt.ylabel('Cantidad Personas')
plt.title('Cantidad de personas por edad')
plt.grid()
plt.show()
plt.close()

##### Eliminacion de edades no validas
    Devido al primer resultado del analisis usando la grafica se presentaron rangos de edad que no pertenercen al grupo
    edades permitidas para optener la licencia de conducion se modifico el rango de edad permitido bajo los criterios que de
    edad para conducir.
    
    - como valor minimo 15 debido a que es la edad minima para obtener la licencia de conduccion.
    - como valor maximo 70 debido que despues de esta edad se solicita renova la licencia de conduccion anualmente. 

In [ ]:
dfsiniestros = dfsiniestros[(dfsiniestros['EDAD_PROCESADA'] >= 15) & (dfsiniestros['EDAD_PROCESADA'] <= 70)]

In [ ]:
gpedad1 = dfsiniestros.groupby('EDAD_PROCESADA')['EDAD_PROCESADA'].count().reset_index(name='CANTIDAD')

In [ ]:
plt.figure(figsize=(15,4))
plt.plot(gpedad1['EDAD_PROCESADA'],gpedad1['CANTIDAD'], 'ro')
"""lstcount = gpedad1['CANTIDAD'].to_numpy()
lstcount1 = gpedad1['EDAD_PROCESADA'].to_numpy()
for i in lstcount1:
    plt.annotate(lstcount[i], (i+0.10, lstcount[i]), color='r')"""
plt.xlabel('Edades')
plt.ylabel('Cantidad Personas')
plt.title('Cantidad de Personas por Edad.')
plt.grid()
plt.show()
plt.close()

### Accidente por mes del año

In [ ]:
#Analisi de personas involucradas en un isniestro por mes en cada año
gpmes17 = dfsiniestros.groupby([dfsiniestros[dfsiniestros['FECHA'].dt.year == 2017]['FECHA'].dt.month])['EDAD_PROCESADA'].count().reset_index(name='CANTIDAD')
gpmes18 = dfsiniestros.groupby([dfsiniestros[dfsiniestros['FECHA'].dt.year == 2018]['FECHA'].dt.month])['EDAD_PROCESADA'].count().reset_index(name='CANTIDAD')
gpmes19 = dfsiniestros.groupby([dfsiniestros[dfsiniestros['FECHA'].dt.year == 2019]['FECHA'].dt.month])['EDAD_PROCESADA'].count().reset_index(name='CANTIDAD')


In [ ]:
plt.figure(figsize=(15,7))
mes = np.array(['ENERO','FEBRERO','MARZO','ABRIL','MAYO','JUNIO','JULIO','AGOSTO','SEPTIEMBRE','OCTUBRE','NOVIEMBRE','DICIEMBRE'])
plt.plot(mes, gpmes17['CANTIDAD'], 'ro-')
plt.plot(mes, gpmes18['CANTIDAD'], 'go-')
plt.plot(mes, gpmes19['CANTIDAD'], 'bo-')
plt.legend(['Año: 2017','Año: 2018','Año: 2019'])
plt.xlabel('Meses del año.')
plt.ylabel('Cantidad de personas.')
plt.title('Cantidad de personas involucradas en siniestros por mes en cada año.')
plt.grid()
plt.show()
plt.close()

#### Grafico 2:

In [ ]:
#Cantidad de siniestros por dia de semana en cada año.
#gpedad = dfsiniestros.groupby('DIA_PROCESADO')['DIA_PROCESADO'].count().reset_index(name='CANTIDAD')
gp17dia = dfsiniestros.groupby([dfsiniestros[dfsiniestros['FECHA'].dt.year == 2017]['FECHA'].dt.year, 'DIA_PROCESADO'])['DIA_PROCESADO'].count().reset_index(name='CANTIDAD')
gp18dia = dfsiniestros.groupby([dfsiniestros[dfsiniestros['FECHA'].dt.year == 2018]['FECHA'].dt.year, 'DIA_PROCESADO'])['DIA_PROCESADO'].count().reset_index(name='CANTIDAD')
gp19dia = dfsiniestros.groupby([dfsiniestros[dfsiniestros['FECHA'].dt.year == 2019]['FECHA'].dt.year, 'DIA_PROCESADO'])['DIA_PROCESADO'].count().reset_index(name='CANTIDAD')

In [ ]:
plt.figure(figsize=(15,7))
dias = np.array(['LUNES','MARTES','MIERCOLES','JUEVES','VIERNES','SÁBADO','DOMINGO'])
plt.plot(dias, gp17dia['CANTIDAD'], 'r.-')
plt.plot(dias, gp18dia['CANTIDAD'], 'g.-')
plt.plot(dias, gp19dia['CANTIDAD'], 'b.-')
lstcant17 = gp17dia['CANTIDAD'].to_numpy()
lstcant18 = gp18dia['CANTIDAD'].to_numpy()
lstcant19 = gp19dia['CANTIDAD'].to_numpy()
for i in range(len(dias)):
    plt.annotate(lstcant17[i], (i+0.10, lstcant17[i]), color='r')
    plt.annotate(lstcant18[i], (i-0.25, 200+lstcant18[i]), color='g')
    plt.annotate(lstcant19[i], (i-0.20, lstcant19[i]-100), color='b')
plt.xlabel('Dias de la Semana')
plt.ylabel('Cantidad de Siniestros')
plt.title('Cantidad de Siniestros por Dia de la Semana por Años.')
plt.legend([2017,2018,2019])
plt.grid()
plt.show()
plt.close()

In [ ]:
g17ploc = dfsiniestros.groupby([dfsiniestros[dfsiniestros['FECHA'].dt.year == 2017]['FECHA'].dt.year,'LOCALIDAD'])['LOCALIDAD'].count().reset_index(name='CANTIDAD')
g18ploc = dfsiniestros.groupby([dfsiniestros[dfsiniestros['FECHA'].dt.year == 2018]['FECHA'].dt.year,'LOCALIDAD'])['LOCALIDAD'].count().reset_index(name='CANTIDAD')
g19ploc = dfsiniestros.groupby([dfsiniestros[dfsiniestros['FECHA'].dt.year == 2019]['FECHA'].dt.year,'LOCALIDAD'])['LOCALIDAD'].count().reset_index(name='CANTIDAD')
localidades = np.array(['KENNEDY','USAQUEN','ENGATIVA','SUBA','FONTIBON','PUENTE ARANDA','CHAPINERO','BARRIOS UNIDOS','TEUSAQUILLO','BOSA','CIUDAD BOLIVAR','LOS MARTIRES','SANTA FE','TUNJUELITO','SAN CRISTOBAL','RAFAEL URIBE URIBE','ANTONIO NARIÑO','USME','CANDELARIA','SUMAPAZ'])

In [ ]:
plt.figure(figsize=(15,4))
plt.plot(localidades,g17ploc['CANTIDAD'], 'r.-')
plt.plot(localidades,g18ploc['CANTIDAD'], 'g.-')
plt.plot(localidades,g19ploc['CANTIDAD'], 'b.-')
plt.xticks(rotation=35)
plt.legend([2017,2018,2019])
plt.xlabel('Localidades Bogotá')
plt.ylabel('Cantidad Siniestros')
plt.title('Cantidad de Siniestros por Localdidad por Años.')
plt.grid()
plt.show()
plt.close()

### Analisis de clase de vehiculo

In [ ]:
gpmescar17 = dfsiniestros.groupby([dfsiniestros[dfsiniestros['FECHA'].dt.year == 2017]['FECHA'].dt.month,'CLASEVEHICULO' ])['CLASEVEHICULO'].count().reset_index(name='CANTIDAD')
gpmescar18 = dfsiniestros.groupby([dfsiniestros[dfsiniestros['FECHA'].dt.year == 2018]['FECHA'].dt.month,'CLASEVEHICULO' ])['CLASEVEHICULO'].count().reset_index(name='CANTIDAD')
gpmescar19 = dfsiniestros.groupby([dfsiniestros[dfsiniestros['FECHA'].dt.year == 2019]['FECHA'].dt.month,'CLASEVEHICULO' ])['CLASEVEHICULO'].count().reset_index(name='CANTIDAD')

In [ ]:
gpmescar17['CLASEVEHICULO'].count

### Analisis uso de elementos de seguridad
#### Uso de chaleco

In [ ]:
# Cantidad de Personas por uso de casco
gpedadlca = dfsiniestros.groupby(['LLEVACASCO','EDAD_PROCESADA'])['EDAD_PROCESADA'].count().reset_index(name='CANTIDAD')
gpelca0 = gpedadlca[gpedadlca['LLEVACASCO']== 0]
gpelca1 = gpedadlca[gpedadlca['LLEVACASCO']== 1]
gpelca2 = gpedadlca[gpedadlca['LLEVACASCO']== 2]

In [ ]:
plt.figure(figsize=(15,4))
plt.plot(gpelca1['EDAD_PROCESADA'],gpelca1['CANTIDAD'], 'ro')
plt.plot(gpelca2['EDAD_PROCESADA'],gpelca2['CANTIDAD'], 'bo', alpha=0.7)
#plt.plot(gpedadcha0['EDAD_PROCESADA'],gpedadcha0['CANTIDAD'], 'go', alpha=0.7)
#plt.xticks(rotation=35)
plt.legend(['USA CINTURÓN','NO USA CINTURÓN'])
plt.xlabel('Edades')
plt.ylabel('Cantidad Personas')
plt.title('Cantidad de Personas por Uso Cinturón.')
plt.grid()
plt.show()
plt.close()

#### Uso de chaleco

In [ ]:
# Cantidad de Personas por uso de chaleco
gpedad = dfsiniestros.groupby(['LLEVACHALECO','EDAD_PROCESADA'])['EDAD_PROCESADA'].count().reset_index(name='CANTIDAD')
gpedadcha0 = gpedad[gpedad['LLEVACHALECO']== 0]
gpedadcha1 = gpedad[gpedad['LLEVACHALECO']== 1]
gpedadcha2 = gpedad[gpedad['LLEVACHALECO']== 2]

In [ ]:
plt.figure(figsize=(15,4))
plt.plot(gpedadcha1['EDAD_PROCESADA'],gpedadcha1['CANTIDAD'], 'ro')
plt.plot(gpedadcha2['EDAD_PROCESADA'],gpedadcha2['CANTIDAD'], 'bo', alpha=0.7)
#plt.plot(gpedadcha0['EDAD_PROCESADA'],gpedadcha0['CANTIDAD'], 'go', alpha=0.7)
#plt.xticks(rotation=35)
plt.legend(['USA CHALECO','NO USA CHALECO'])
plt.xlabel('Edades')
plt.ylabel('Cantidad Personas')
plt.title('Cantidad de Personas por Uso Chaleco.')
plt.grid()
plt.show()
plt.close()

#### Uso cinturón

In [ ]:
gpedad = dfsiniestros.groupby(['LLEVACINTURON','EDAD_PROCESADA'])['EDAD_PROCESADA'].count().reset_index(name='CANTIDAD')
gpelc0 = gpedad[gpedad['LLEVACINTURON']== 0]
gpelc1 = gpedad[gpedad['LLEVACINTURON']== 1]
gpelc2 = gpedad[gpedad['LLEVACINTURON']== 2]

In [ ]:
plt.figure(figsize=(15,4))
plt.plot(gpelc1['EDAD_PROCESADA'],gpelc1['CANTIDAD'], 'ro')
plt.plot(gpelc2['EDAD_PROCESADA'],gpelc2['CANTIDAD'], 'bo', alpha=0.7)
#plt.plot(gpedadcha0['EDAD_PROCESADA'],gpedadcha0['CANTIDAD'], 'go', alpha=0.7)
#plt.xticks(rotation=35)
plt.legend(['USA CINTURÓN','NO USA CINTURÓN'])
plt.xlabel('Edades')
plt.ylabel('Cantidad Personas')
plt.title('Cantidad de Personas por Uso Cinturón.')
plt.grid()
plt.show()
plt.close()

### Analisis de gravedad del siniestro por gravedad de personas involucradas.
#### GRAVEDADCOD
1. CON MUERTOS
2. CON HERIDOS
3. SOLO DAÑOS

#### GRAVEDAD_PROCESADA
1. ILESA
2. HERIDO VALORADO
3. HERIDO HOSPITALIZADO
4. MUERTA

In [ ]:
# Cantidad de siniestros por gravedades
gpgrav = dfsiniestros.groupby(['GRAVEDAD_PROCESADA','GRAVEDADCOD'])['EDAD_PROCESADA'].count().reset_index(name='CANTIDAD')
gpgrav

- 1  1 = Muertos  ```4```
- 1  2 = Heridos Valorado ```2```
- 1  3 = Ilesos  ```1```
- 2  1 = Heridos Hospitalizado ```3```
- 2  2 = Heridos Valorado  ```2```
- 2  3 = Ilesos  ```1```
- 3  1 = Heridos Hospitalizado  ```3```
- 3  2 = Heridos Hospitalizado  ```3```
- 4  1 = Muertos  ```4```

In [ ]:
# Creación de campo gravedad con datos genericos
dfsiniestros['GRAVEDAD'] = dfsiniestros['GRAVEDADCOD']

In [ ]:
# Muertos 1 1
dfsiniestros.loc[( (dfsiniestros['GRAVEDAD_PROCESADA'] == 1) & (dfsiniestros['GRAVEDADCOD'] == 1) ),'GRAVEDAD'] = 4 
# Muertos 1 2
dfsiniestros.loc[( (dfsiniestros['GRAVEDAD_PROCESADA'] == 1) & (dfsiniestros['GRAVEDADCOD'] == 2) ),'GRAVEDAD'] = 2
# Muertos 1 3
dfsiniestros.loc[( (dfsiniestros['GRAVEDAD_PROCESADA'] == 1) & (dfsiniestros['GRAVEDADCOD'] == 3) ),'GRAVEDAD'] = 1 
# Muertos 2 1
dfsiniestros.loc[( (dfsiniestros['GRAVEDAD_PROCESADA'] == 2) & (dfsiniestros['GRAVEDADCOD'] == 1) ),'GRAVEDAD'] = 3 
# Muertos 2 2
dfsiniestros.loc[( (dfsiniestros['GRAVEDAD_PROCESADA'] == 2) & (dfsiniestros['GRAVEDADCOD'] == 2) ),'GRAVEDAD'] = 2 
# Muertos 2 3
dfsiniestros.loc[( (dfsiniestros['GRAVEDAD_PROCESADA'] == 2) & (dfsiniestros['GRAVEDADCOD'] == 3) ),'GRAVEDAD'] = 1 
# Muertos 3 1
dfsiniestros.loc[( (dfsiniestros['GRAVEDAD_PROCESADA'] == 3) & (dfsiniestros['GRAVEDADCOD'] == 1) ),'GRAVEDAD'] = 3 
# Muertos 3 2
dfsiniestros.loc[( (dfsiniestros['GRAVEDAD_PROCESADA'] == 3) & (dfsiniestros['GRAVEDADCOD'] == 2) ),'GRAVEDAD'] = 3 
# Muertos 4 1
dfsiniestros.loc[( (dfsiniestros['GRAVEDAD_PROCESADA'] == 4) & (dfsiniestros['GRAVEDADCOD'] == 1) ),'GRAVEDAD'] = 4 

In [ ]:
# Cantidad de siniestros agrupados por tipo de gravedad de accidentes.
gpgrav = dfsiniestros.groupby(['GRAVEDAD'])['GRAVEDAD'].count().reset_index(name='CANTIDAD')

In [ ]:
# Mostramos la cantidad de siniestros por gravedad.
plt.figure(figsize=(15,4))
lstgrave = np.array(['ILESOS','HERIDO VALORADO','HERIDO HOSPITALIZADO','MUERTOS'])
plt.bar(lstgrave, gpgrav['CANTIDAD'], width=0.9, color='#FF616D')
plt.title("Cantidad de dfsiniestros por gravedad.")
plt.ylabel("Cantidad Siniestros.")
plt.xlabel("Gravedad Siniestro.")
plt.xticks(rotation=20)
lstcant = gpgrav['CANTIDAD'].to_numpy()

for i in range(len(lstcant)):
    plt.annotate(lstcant[i], (i-0.10, 1000+lstcant[i]))

plt.show()
plt.close()

In [ ]:
# Cantidad de siniestros agrupados por tipo de gravedad de accidentes.
gpgran = dfsiniestros.groupby([dfsiniestros['FECHA'].dt.year,'GRAVEDAD'])['GRAVEDAD'].count().reset_index(name='CANTIDAD')
#gpgran

In [ ]:
gpgra17 = gpgran[gpgran['FECHA'] == 2017]
gpgra18 = gpgran[gpgran['FECHA'] == 2018]
gpgra19 = gpgran[gpgran['FECHA'] == 2019]
gpgra19

In [ ]:
plt.figure(figsize=(15,6))
lstgravedad = np.array(['ILESOS','HERIDO VALORADO','HERIDO HOSPITA.','MUERTOS'])
barWidth = 0.25
r1 = np.arange(len(lstgravedad))
r2 = [x + barWidth for x in r1]
r3 = [x + barWidth for x in r2]
plt.bar(r1,gpgra17['CANTIDAD'], width=barWidth, color='red')
plt.bar(r2,gpgra18['CANTIDAD'], width=barWidth,color='green')
plt.bar(r3,gpgra19['CANTIDAD'], width=barWidth, color='blue')
plt.legend(['Año 2017','Año 2018', 'Año 2019'])
# Add xticks on the middle of the group bars
plt.xlabel('group', fontweight='bold')
plt.xticks([r + barWidth for r in range(len(lstgravedad))], ['ILESOS','HERIDO VALORADO','HERIDO HOSPITALIZADO','MUERTOS'], rotation=20)
lstcant = gpgra17['CANTIDAD'].to_numpy()
for i in range(len(lstcant)):
    plt.annotate(lstcant[i], (i-0.10, 500+lstcant[i]))
    
lstcant18 = gpgra18['CANTIDAD'].to_numpy()    
for i in range(len(lstcant18)):
    plt.annotate(lstcant18[i], (i+0.18, 500+lstcant18[i]))
    
lstcant19 = gpgra19['CANTIDAD'].to_numpy()
for i in range(len(lstcant19)):
    plt.annotate(lstcant19[i], (i+0.45, 500+lstcant19[i]))
plt.xlabel('Gravedades')    
plt.ylabel('Cantidad Siniestros')
plt.title('Cantidad de Siniestros por gravedades.')
plt.show()
plt.close()

In [ ]:
gracogpcoun = dfsiniestros.groupby(['GRAVEDAD_PROCESADA']).size().reset_index(name='CANTIDAD')
gracogpcoun = gracogpcoun.sort_values(by=['CANTIDAD'], ascending=True)
#gracogpcoun

In [ ]:
plt.figure()
lstgrave = np.array(['MUERTOS','HERIDO HOSPITA.','HERIDO VALORADO','ILESOS'])
plt.bar(lstgrave, gracogpcoun['CANTIDAD'], width=0.7, color='#053742')
plt.title("Cantidad de dfsiniestros por gravedad.")
plt.ylabel("Gravedad dfsiniestros.")
plt.xlabel("Cantidad Siniestro.")
plt.xticks(rotation=90)
lstcant = gracogpcoun['CANTIDAD'].to_numpy()

for i in range(len(lstcant)):
    plt.annotate(lstcant[i], (i-0.15, 2000+lstcant[i]))

plt.show()
plt.close()

### Campo de pruebas 

In [ ]:
dfsiniestros.columns

In [ ]:
dfsiniestros.drop('DIRECCION', axis=1).to_csv(URL_PREPARED_DATA+'/siniestros-train.csv')

In [ ]:
df['Date'] = df['Date'].astype('datetime64[ns]')

In [ ]:
dfpruebba = dfsiniestros.copy()

In [ ]:
for column in dfsiniestros.columns:
    print(column)
    print(dfsiniestros[column].value_counts(dropna=False))

In [ ]:
dfsiniestros['EDAD_PROCESADA'] = dfsiniestros['EDAD_PROCESADA'].replace('SIN INFORMACION',round(eda.mean(),0))

In [ ]:
dfsiniestros['EDAD_PROCESADA'].map(type).value_counts()

In [ ]:
round(eda.mode().get(0),0)

In [ ]:
dfsiniestros['EDAD_PROCESADA'].fillna(round(eda.mode().get(0),0), inplace=True)

In [ ]:
for column in dfsiniestros.columns:
    print(f'Columna {column} tienen {dfsiniestros[column].isna().sum()} registros en nan')

In [ ]:
dfsiniestros['EDAD_PROCESADA'] = dfsiniestros['EDAD_PROCESADA'].astype(float)

In [ ]:
dfsiniestros['EDAD_PROCESADA'].map(type).value_counts()

In [ ]:
graedadagrup = dfsiniestros.groupby(['EDAD_PROCESADA']).size().reset_index(name='CANTIDAD')

### Cantidad de dfsiniestros por gravedad.

In [ ]:
gragpcoun = dfsiniestros.groupby(['GRAVEDADCOD']).size().reset_index(name='CANTIDAD')
gragpcoun

In [ ]:
lstgrave = ['Con Muertos','Con Heridos','Solo Daños']

In [ ]:
plt.figure()
plt.bar(lstgrave, gragpcoun['CANTIDAD'], width=0.5, color='#FF616D')
plt.title("Cantidad de dfsiniestros por gravedad.")
plt.ylabel("Cantidad dfsiniestros.")
plt.xlabel("Gravedad Siniestro.")
plt.xticks(rotation=90)
lstcant = gragpcoun['CANTIDAD'].to_numpy()

for i in range(len(lstcant)):
    plt.annotate(lstcant[i], (i-0.15, 2000+lstcant[i]))

plt.show()
plt.close()

In [ ]:
dfsiniestros[['GRAVEDAD_PROCESADA','GRAVEDADCOD']].value_counts()

In [ ]:
dfsiniestros[['EDAD_PROCESADA','GRAVEDADCOD']].value_counts()

In [ ]:
graedadagrup = dfsiniestros.groupby(['EDAD_PROCESADA','GRAVEDADCOD']).size().reset_index(name='CANTIDAD')
graedadagrup

In [ ]:
plt.figure()
plt.plot(graedadagrup['EDAD_PROCESADA'], graedadagrup['CANTIDAD'],'g-')
plt.xlabel('Edad Procesada')
plt.ylabel('Cantidad')
plt.savefig('pruebaas')
plt.show()
plt.close()

In [ ]:
dfsiniestros.info()